<a href="https://colab.research.google.com/github/ABHISHEK-DEY-64150/Bioinformatics/blob/main/2017_CSE470_2017331021_E09.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## BA7A : Compute Distances Between Leaves

In [ ]:
!pip install networkx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import networkx as nx

def dist_matr(weighted_edge_list):
  tree = nx.Graph()
  tree.add_weighted_edges_from(weighted_edge_list)
 
  dist = dict(nx.all_pairs_dijkstra_path_length(tree))
  leaves = [n for n in tree.nodes() if tree.degree(n)==1]
  return [[dist[n1][n2] for n1 in leaves] for n2 in leaves]


In [ ]:

n = 4

input_txt = '''0->4:11
1->4:2
2->5:6
3->5:7
4->0:11
4->1:2
4->5:4
5->4:4
5->3:7
5->2:6'''.replace('->',' ').replace(':',' ').split()


weighted_edge_list = [(u,v,int(w)) for u,v,w in zip(input_txt[::3], input_txt[1::3], input_txt[2::3]) ]

dist = dist_matr(weighted_edge_list)
print('\n'.join([' '.join([str(d) for d in row]) for row in dist]))



0 13 21 22
13 0 12 13
21 12 0 13
22 13 13 0


## BA7B : Compute Limb Lengths in a Tree

In [ ]:
import numpy as np
import itertools as it

def limb_length(dist_m,n,i):
  return min((dist_m[i,j] + dist_m[i,k] - dist_m[j,k])//2 for j,k in 
             it.combinations(it.chain(range(i),range(i+1,n)),2))


n = 4
i = 1

input_matrix = '''0   13  21  22
13  0   12  13
21  12  0   13
22  13  13  0'''

dist_m = np.fromstring(input_matrix,dtype = int,sep = ' ').reshape(n,n)
limb_length(dist_m,n,i)


2

## BA7C : Implement AdditivePhylogeny

In [ ]:
def parent_dist(dm,n):
    for i,k in it.combinations(range(n-1),2):
        if dm[i,k]==dm[i,n-1]+dm[n-1,k]:
            return i,k
    return 0,0
def additive_phylogeny(dm,n):
    def additive_phylogeny_r(dm,n,v):
        if n==2:
            return {0:{1:dm[0,1]},1:{0:dm[1,0]}}
        limb_len = limb_length(dm,n,n-1)
        dm[n-1],dm[:,n-1] = dm[n-1]-limb_len,dm[:,n-1]-limb_len
        i,k = parent_dist(dm,n)
        x = dm[i,n-1]
        tree = additive_phylogeny_r(np.delete(np.delete(dm,n-1,1),n-1,0),n-1,v+1)
        path = nx.shortest_path(nx.Graph(tree),i,k)
        dist,pathi = 0,0
        while tree[path[pathi]][path[pathi+1]]+dist<x:
            dist += tree[path[pathi]][path[pathi+1]]
            pathi += 1
        i,k,x = path[pathi],path[pathi+1],x-dist
        if x==0: v=k
        tree[i][v],tree[k][v] = x,tree[i][k]-x
        del tree[i][k]
        del tree[k][i]
        tree[n-1] = {v:limb_len}
        tree[v] = {n-1:limb_len,i:x,k:tree[k][v]}
        return tree
    return additive_phylogeny_r(dm,n,n)

n = 4
input_str = '''0   13  21  22
13  0   12  13
21  12  0   13
22  13  13  0'''
dm = np.fromstring(input_str,dtype=int,sep=' ').reshape(n,n)
phylo_tree = additive_phylogeny(dm,n)
print('\n'.join(f'{key}->{val}:{weight}' for key,value in sorted(phylo_tree.items()) for val,weight in value.items()))

0->5:11
1->5:2
2->4:6
3->4:7
4->3:7
4->5:4
4->2:6
5->0:11
5->1:2
5->4:4
